In [1]:
using JuMP, HiGHS

In [2]:
m = Model(HiGHS.Optimizer)

@variable(m, 0 <= x)
@variable(m, 0 <= y)
@variable(m, 0 <= z)

@constraint(m, 2x + 3y + 4z <= 120.0)
@constraint(m, x + 2y <= 50.0)
@constraint(m, x + 2z <= 50.0)

@objective(m, Min, -2x-y-3z)

print(m)

In [3]:
status = optimize!(m)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3 rows, 3 cols, 7 nonzeros
3 rows, 3 cols, 7 nonzeros
Presolve : Reductions: rows 3(-0); columns 3(-0); elements 7(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -5.9999935790e+00 Ph1: 3(15); Du: 3(5.99999) 0s
          3    -1.0000000000e+02 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 3
Objective value     : -1.0000000000e+02
HiGHS run time      :          0.00


In [4]:
[value(x), value(y), value(z)]

3-element Vector{Float64}:
 50.0
  0.0
 -0.0

In [5]:
function pivot!(M::Matrix, i::Int, j::Int)
    m, n = size(M)
    @assert M[i, j] != 0
    M[i, :] = M[i, :]/M[i, j]
    for k in setdiff(1:m, i)
        M[k, :] -= M[k, j] * M[i, :]
    end
    return M
end

function getReducedCosts(M::Matrix)
    m, n = size(M)
    return M[end, 1:n-1]
end
function getxB(M::Matrix)
    m, n = size(M)
    return M[1:m-1, end]
end
function enteringVar(M::Matrix)
    rc = getReducedCosts(M)
    index = argmin(rc)
    return rc[index] >= 0 ? -1 : index
end
function exitingVarIndex(M::Matrix{T}, enteringVar::Int) where T
    col = M[1:end-1, enteringVar]
    xB = getxB(M)
    m, n = size(M)
    index = -1
    val = T(Inf)
    for i in 1:m-1
        if (col[i] > 0) && (xB[i]/col[i] < val)
            val = xB[i]/col[i]
            index = i
        end
    end
    return index
end
function isOneHot(v::Vector)
    n = length(v)
    return (sum(iszero, v) == n-1) && (sum(isone, v) == 1)
end 
function isoptimal(M)
    return enteringVar(M) == -1
end
function findInitialBasis!(M::Matrix)
    m, n = size(M)
    m-=1
    n-=1
    basis = [-1 for _ in 1:m]
    for i in 1:n
        if isOneHot(M[1:end-1, i])
            index = findfirst(isone, M[:, i])
            basis[index] = i
        end
    end
    @assert !any(t-> t == -1, basis) "problem not caconical"
    for i in 1:m
        j = basis[i]
        pivot!(M, i, j)
    end
    return basis
end
function simplexSolver(A::Matrix{T}, b::Vector, c::Vector; verbose::Bool = false) where T
    M = [A b; c' 0]
    basis = findInitialBasis!(M)
    k = 1
    nmax = 1000
    while !isoptimal(M) && k < nmax
        k+=1
        verbose && display(M)
        entering = enteringVar(M)
        entering == -1 && (println("------------"); return T[-1, -1])
        leaving = exitingVarIndex(M, entering)
        leaving == -1 && (println("------------"); return T[-1, -1])
        verbose && @show (entering, leaving)
        basis[leaving] = entering
        pivot!(M, leaving, entering)
    end
    verbose && display(M)
    m, n = size(M)
    xstar = zeros(T, n - 1)
    xstar[basis] = getxB(M)
    return xstar
end

simplexSolver (generic function with 1 method)

In [6]:
A=[2//1 3 4 1 0 0 ; 
    1 2 0 0 1 0;
    1 0 2 0 0 1;
]
b=[120, 50, 50]
c = [-2,-1,-3,0,0,0]

6-element Vector{Int64}:
 -2
 -1
 -3
  0
  0
  0

In [7]:
simplexSolver(A, b, c, verbose=true)

4×7 Matrix{Rational{Int64}}:
  2//1   3//1   4//1  1//1  0//1  0//1  120//1
  1//1   2//1   0//1  0//1  1//1  0//1   50//1
  1//1   0//1   2//1  0//1  0//1  1//1   50//1
 -2//1  -1//1  -3//1  0//1  0//1  0//1    0//1

(entering, leaving) = (3, 3)


4×7 Matrix{Rational{Int64}}:
  0//1   3//1  0//1  1//1  0//1  -2//1  20//1
  1//1   2//1  0//1  0//1  1//1   0//1  50//1
  1//2   0//1  1//1  0//1  0//1   1//2  25//1
 -1//2  -1//1  0//1  0//1  0//1   3//2  75//1

(entering, leaving) = (2, 1)


4×7 Matrix{Rational{Int64}}:
  0//1  1//1  0//1   1//3  0//1  -2//3   20//3
  1//1  0//1  0//1  -2//3  1//1   4//3  110//3
  1//2  0//1  1//1   0//1  0//1   1//2   25//1
 -1//2  0//1  0//1   1//3  0//1   5//6  245//3

(entering, leaving) = (1, 2)


4×7 Matrix{Rational{Int64}}:
 0//1  1//1  0//1   1//3   0//1  -2//3   20//3
 1//1  0//1  0//1  -2//3   1//1   4//3  110//3
 0//1  0//1  1//1   1//3  -1//2  -1//6   20//3
 0//1  0//1  0//1   0//1   1//2   3//2  100//1

6-element Vector{Rational{Int64}}:
 110//3
  20//3
  20//3
   0//1
   0//1
   0//1